In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fmin_cobyla

## (a)

In [2]:
def compute_arrow_prices(A, p):
    # Transpose A to get assets x states
    A_T = A.T
    # Solve the linear system A_T * p_bar = p
    arrow_prices = np.linalg.solve(A_T, p)
    return arrow_prices

# Given payoff matrix A and prices p
A = np.array([
    [5, 30, 5],
    [5, 10, 0],
    [5, 0, 10]
])

p = np.array([1, 2, 1])

# Compute Arrow prices
arrow_prices = compute_arrow_prices(A, p)

print("Arrow-Debreu Prices:")
for i, price in enumerate(arrow_prices):
    print(f"p̄_{i} = {price:.4f}")

Arrow-Debreu Prices:
p̄_0 = 0.0400
p̄_1 = 0.0800
p̄_2 = 0.0800


## (b)

In [3]:
def check_asset_mispricing(A_new, p_new, arrow_prices):

    # Compute theoretical price
    p_star = np.dot(A_new, arrow_prices)
    # Determine mispricing
    if p_star > p_new:
        status = "The asset is too cheap (underpriced)."
        arbitrage_profit = p_star - p_new
    elif p_star < p_new:
        status = "The asset is too expensive (overpriced)."
        arbitrage_profit = p_new - p_star
    else:
        status = "The asset is correctly priced."
        arbitrage_profit = 0.0
    return p_star, status, arbitrage_profit

# Given new asset A3 and price p3
A3 = np.array([2, 2, 3])
p3 = 4

# Check mispricing
p_star, status, arbitrage_profit = check_asset_mispricing(A3, p3, arrow_prices)

print(status)
print(f"Sell 1 unit of asset 3 at ${p3} and buy the portfolio replication (2 unit of p̄0, 2 units of p̄1, 3 units of p̄2) for ${p_star:.2f}")
print(f"Arbitrage profit per unit: {arbitrage_profit:.2f}")


The asset is too expensive (overpriced).
Sell 1 unit of asset 3 at $4 and buy the portfolio replication (2 unit of p̄0, 2 units of p̄1, 3 units of p̄2) for $0.48
Arbitrage profit per unit: 3.52


## (c)

In [4]:

# Given data
A = np.array([
    [5, 30, 5],
    [5, 10, 0],
    [5, 0, 10]
], dtype=float)

p = np.array([1, 2, 1], dtype=float)

# Arrow-Debreu prices computed from Part a)
p_bar = np.array([0.04, 0.08, 0.08])  # [0.0400, 0.0800, 0.0800]

# Probabilities of each state
pi = np.array([0.2, 0.3, 0.5])

theta = 4  # Given theta
m = 2000   # Jake's wealth

# Compute k_t = (pi_t / p_bar_t)^{1/theta}
k_t = (pi / p_bar) ** (1 / theta)

# Compute D = sum over t of (pi_t^{1/theta} * p_bar_t^{(theta - 1)/theta})
pi_power = pi ** (1 / theta)
p_bar_power = p_bar ** ((theta - 1) / theta)
D = np.sum(pi_power * p_bar_power)

# Compute m / D
m_over_D = m / D

# Compute x_t = k_t * (m / D)
x_t = k_t * m_over_D

print("Optimal Consumption x_t:")
for t in range(len(x_t)):
    print(f"x_{t} = {x_t[t]:.4f}")

# Check if Jake fully hedges his risk
if np.allclose(x_t, x_t[0]):
    print("\nJake fully hedges his risk; consumption is the same across all states.")
else:
    print("\nJake does not fully hedge his risk; consumption varies across states.")

Optimal Consumption x_t:
x_0 = 10048.3279
x_1 = 9351.0228
x_2 = 10624.8132

Jake does not fully hedge his risk; consumption varies across states.


## (d)

In [5]:
# Solve for omega
omega = np.linalg.solve(A, x_t)

print("\nOptimal Portfolio omega_n (Quantities of Assets):")
for n in range(len(omega)):
    print(f"omega_{n} = {omega[n]:.4f}")

# Verify the portfolio generates the desired consumption x_t
x_t_check = A.dot(omega)
print("\nVerification: A * omega = x_t (Should match the computed x_t)")
for t in range(len(x_t_check)):
    print(f"State {t}: x_t = {x_t[t]:.4f}, A * omega = {x_t_check[t]:.4f}")

# Verify the budget constraint with asset prices
total_portfolio_cost = np.dot(p, omega)
print(f"\nTotal portfolio cost: {total_portfolio_cost:.2f} (Jake's Wealth m = {m})")




Optimal Portfolio omega_n (Quantities of Assets):
omega_0 = 1865.3718
omega_1 = 2.4164
omega_2 = 129.7954

Verification: A * omega = x_t (Should match the computed x_t)
State 0: x_t = 10048.3279, A * omega = 10048.3279
State 1: x_t = 9351.0228, A * omega = 9351.0228
State 2: x_t = 10624.8132, A * omega = 10624.8132

Total portfolio cost: 2000.00 (Jake's Wealth m = 2000)
